In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
if not os.path.exists('./certs'):
    os.mkdir('certs')

In [3]:
os.listdir()

['.idea',
 '.ipynb_checkpoints',
 'ALX-T Connect Weekly Schedule - Data.pdf',
 'certs',
 'cleaning_and_splitting_movie_genres.ipynb',
 'connect_sessions_studyguide.pdf',
 'data-wrangling-template.ipynb',
 'datasets',
 'data_visualization.ipynb',
 'data_wrangling.ipynb',
 'downloadng_ebert_images.ipynb',
 'ebert_imgs.csv',
 'extract_attendance_from_cloudSQL.ipynb',
 'img',
 'img_errors.json',
 'lesson3_the_data_analysis_process.ipynb',
 'load_students_table_cloudsql.ipynb',
 'media_wiki.ipynb',
 'merging-data.ipynb',
 'name.png',
 'testing.ipynb',
 'web_scraping.ipynb',
 'wine_visualizations_solutions.ipynb']

In [4]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [5]:
config = {
    'user': 'root',
    'password': 'roots',
    'host': '34.133.227.55',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'certs/server-ca.pem',
    'ssl_cert': 'certs/client-cert.pem',
    'ssl_key': 'certs/client-key.pem'
}

In [6]:
# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [7]:
if cnxn.is_connected():
    # Get connection info
    db_Info = cnxn.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    # Initiaize a cursor object
    cursor = cnxn.cursor()

Connected to MySQL Server version  8.0.26-google


In [8]:
cursor.execute('USE alxt')

In [9]:
cursor.execute("SHOW TABLES;")
out = cursor.fetchall()
out

[('att_07_09',), ('students',)]

In [10]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 60) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [11]:
query = 'SELECT * FROM att_07_09;'
attendance = query_to_df(query)

Query ran for 0.7001125812530518 secs!


In [12]:
# sort attendance alphabetically, and replace index

attendance = attendance.sort_values(by=['first_name', 'last_name']).reset_index(drop=True)

In [13]:
# Next select only those present

present = attendance[attendance['status']==1]

In [14]:
print(len(present))

4


In [15]:
present

,first_name,last_name,email,status,time
19,Emmanuel,Otiti,fammava@gmail.com,1,19:03
20,Eric,Kwabena preko addo,addoericjnr@gmail.com,1,
21,Etuonu,David,davidetuonu12@gmail.com,1,19:02
38,Kelvin,Mwaka,kelvinmwaka@gmail.com,1,19:01


In [ ]:
cursor.close()